In [ ]:
#Import the necessary libraries

import nltk
nltk.download('punkt')
import re
import pandas
import copy
from collections import Counter
from nltk import ngrams
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

#Read in the train.txt file

fp = open('train.txt', 'r')
train = fp.readlines()
fp.close()

#Number of sentences in train.txt that pertain to each word pair
dist = [24200, 24200, 2890, 24200, 24200, 24200, 7269, 24200, 24200, 24200,
       24200, 24200, 6068, 24200, 24200, 3396, 24200, 24200, 24200, 24200, 
       12105, 11497, 16563, 5049, 4534]

#List of tuples of the word pairs
pairs = [
    ("a" , "á"),
    ("ais", "áis"),
    ("aisti", "aistí"),
    ("ait", "áit"),
    ("ar", "ár"),
    ("arsa", "ársa"),
    ("ban", "bán"),
    ("cead", "céad"),
    ("chas", "chás"),
    ("chuig", "chúig"),
    ("dar", "dár"),
    ("do", "dó"),
    ("gaire", "gáire"),
    ("i", "í"),
    ("inar", "inár"),
    ("leacht", "léacht"),
    ("leas", "léas"),
    ("mo", "mó"),
    ("na", "ná"),
    ("os", "ós"),
    ("re", "ré"),
    ("scor", "scór"),
    ("te", "té"),
    ("teann", "téann"),
    ("thoir", "thóir")
]

        
    

In [ ]:
#Code to label each sentence in train.txt as either accented or ascii

pairsindex = 0
count = 0

asciioraccent = []
sector = []

sentences = []
sentencesector = []


for t in train:
    if count == dist[pairsindex]:
        #print(pairs[pairsindex][0], pairs[pairsindex][1])
        pairsindex += 1
        asciioraccent.append(copy.deepcopy(sector))
        sentences.append(copy.deepcopy(sentencesector))
        #print(sector[1:50])
        sector.clear()
        sentencesector.clear()
        count = 0
    sentence = re.compile('\w+').findall(t)
    for s in sentence:
        if s == pairs[pairsindex][0] or s == pairs[pairsindex][1]:
            target = s
            if target == pairs[pairsindex][0]:
                sector.append(1)
            else:
                sector.append(0)
            sentencesector.append(t)
            break
    count += 1

sentences.append(copy.deepcopy(sentencesector))
asciioraccent.append(copy.deepcopy(sector))

sum = 0
for a in asciioraccent:
    sum += len(a)


In [ ]:
#Splitting training set into training and validation sets

training = []
val = []
answers = []

sum = 0
sum2 = 0
sum3 = 0

for i in range(0, 25):
    trainer, validate, ignore, answer = train_test_split(sentences[i], asciioraccent[i])
    training.append(trainer)
    val.append(validate)
    answers.append(answer)
    sum += len(trainer)
    sum2 += len(validate)
    sum3 += len(answer)

In [ ]:
#Dumping all the words in train.txt into 25 big lists - one for each word pair

distindex = 0
count = 0

wordclumps = []
wordclumpdist = []

for t in training:
    for q in t:
        sentence = re.compile('\w+').findall(q)
        for s in sentence:
            wordclumpdist.append(s)
    wordclumps.append(copy.deepcopy(wordclumpdist))
    wordclumpdist.clear()        
    


In [ ]:
#Training algorithm - this involves taking the unigram and bigram of each big list of "wordclumps", finding the bigrams
#that have one of the words in the word pairs in them (bigrams that have either a or á, for example), and computing
#probabilities for those

pairsindex = 0
    
probs = {}

unipair = []
unigrams = []
bigrams = []
pertbigrams = []

possiblepairs = []
distinctseen = []
totalseen = []
unseen = []
onepairs = []

for wc in wordclumps:
    
    uni = list(Counter(wc).items())
    bi = list(Counter(ngrams(wc, 2)).items())
    
    possible = len(uni) * len(uni)
    distinct = len(bi)
    
    total = 0
    for u in uni:
        total += u[1]
        
    un = possible - distinct
    
    one = 0
    for b in bi:
        if b[1] == 1:
            one += 1
    
    possiblepairs.append(possible)
    distinctseen.append(distinct)
    totalseen.append(total)
    unseen.append(un)
    onepairs.append(one)
    
    unigrams = uni
    bigrams = bi
    
    
    
    for b in bigrams:
        if pairs[pairsindex][0] in b[0] or pairs[pairsindex][1] in b[0]:
            pertbigrams.append(b)
    
    for p in pertbigrams:
        otherword = 0
        if p[0][0] == pairs[pairsindex][0] or p[0][0] == pairs[pairsindex][1]:
            for u in unigrams:
                if u[0] == p[0][1]:
                    otherword = u[1]
                    break
            if otherword == 0:
                continue
            else:
                probs[p[0]] = p[1]/otherword
               #print(p[0], p[1]/otherword)
                continue
        if p[0][1] == pairs[pairsindex][0] or p[0][1] == pairs[pairsindex][1]:
            for u in unigrams:
                if u[0] == p[0][0]:
                    otherword = u[1]
                    break
            if otherword == 0:
                continue
            else:
                probs[p[0]] = p[1]/otherword
                #print(p[0], p[1]/otherword)
                continue
                
    
    for u in unigrams:
        if pairs[pairsindex][0] in u or pairs[pairsindex][1] in u:
            unipair.append(u)
    
    pertbigrams = []
    print("Iteration " + str(pairsindex + 1) + " complete")
    pairsindex += 1
        

    
print("Training complete")

In [ ]:
#Dumping all the words in wordclumps into one really big list of words and taking unigrams and probabilities of that really
#big list

wordbag = []
alluniprobs = {}

for wc in wordclumps:
    for w in wc:
        wordbag.append(w)

allwords = len(wordbag)
allunigrams = list(Counter(wc).items())

for a in allunigrams:
    alluniprobs[a[0]] = a[1]/allwords

In [ ]:
#Compute unigram probabilities for each of the word pairs

uniwordprobs = {}

pairsindex = 0
distindex = 0
count = 1

asciicount = 0
accentcount = 0

for t in train:
    count += 1
    sentence = re.compile('\w+').findall(t)
    for s in sentence:
        if s == pairs[pairsindex][0]:
            asciicount += 1
            break
        elif s == pairs[pairsindex][1]:
            accentcount += 1
            break
    if count == dist[distindex]:
        count = 0
        uniwordprobs[pairs[pairsindex][0]] = (asciicount/dist[distindex])
        uniwordprobs[pairs[pairsindex][1]] = (accentcount/dist[distindex])
        if distindex < 24:
            distindex += 1
        if pairsindex < 24:
            pairsindex += 1
        asciicount = 0
        accentcount = 0


In [ ]:
#Replacing word pair words in the sentences in the validation set with the {|} structure
#E.g. replacing "a" in a sentence with {a|á}

pairsindex = 0

validation = []
valsector = []

sum = 0

for v1 in val:
    decider = "{" + pairs[pairsindex][0] + "|" + pairs[pairsindex][1] + "}"
    for v in v1:
        sentence = re.compile('[\w{}À-ÿ|]+').findall(v)
        target = ""
        for s in sentence:
            if s == pairs[pairsindex][0] or s == pairs[pairsindex][1]:
                target = s
                break
        concat = ' '.join(sentence)
        sub = "\\b" + target + "\\b"
        valsector.append(re.sub(sub, decider, concat).strip('\n'))
    sum += len(valsector)
    validation.append(copy.deepcopy(valsector))
    valsector.clear()
    pairsindex += 1

In [ ]:
#Training interpolation hyperparameters (lambdas) on the validation set

pairsindex = 0
guesses = []
allanswers = []


for an in answers:
    for a in an:
        allanswers.append(a)

#front bigram
lamb = 0.35

#front unigram
lamb2 = 0.2

#back bigram
lamb3 = 0.35

#back unigram
lamb4 = 0.1

sum = 0
for vi in validation:
    sum += len(vi)

#print(sum)

reps = 0
    
#Cycle through the validation set, find the {|} structure, and use interpolation to calculate guesses based off of the
#previous word and the next word
for vi in validation:
    decide = "{" + pairs[pairsindex][0] + "|" + pairs[pairsindex][1] + "}"
    for v in vi:
        sentence = re.compile('[\w{}À-ÿ|]+').findall(v)
        for s in sentence:
            if s == decide:
                #find previous word and next word
                prevword, nextword = None, None
                index = sentence.index(decide)
                if index > 0:
                    prevword = sentence[index - 1]
                if index < (len(sentence) - 1):
                    nextword = sentence[index + 1]
                
                #calculate alpha and beta according to interpolation
                alpha = 0
                beta = 0

                afront = 0.0
                aback = 0.0
                bfront = 0.0
                bback = 0.0

                afront = lamb * probs.get((prevword, pairs[pairsindex][0]), 0.0) \
                + lamb2 * uniwordprobs.get(pairs[pairsindex][0], 0.0)
                
                aback = lamb3 * probs.get((pairs[pairsindex][0], nextword), 0.0) \
                + lamb4 * alluniprobs.get(nextword, 0.0)

                bfront = lamb * probs.get((prevword, pairs[pairsindex][1]), 0.0) \
                + lamb2 * uniwordprobs.get(pairs[pairsindex][1], 0.0)
                
                bback = lamb3 * probs.get((pairs[pairsindex][1], nextword), 0.0) \
                + lamb4 * alluniprobs.get(nextword, 0.0)

                alpha =  afront * aback
                beta =  bfront * bback
                if alpha != 0 and beta != 0:
                    guesses.append(alpha/(alpha + beta))
                else:
                    uniprob = uniwordprobs.get(pairs[pairsindex][0], 0.0)
                    guesses.append(uniprob)
                reps += 1
                break
    pairsindex += 1
    

#Take log loss in order to improve hyperparameters    
print(log_loss(allanswers, guesses))

In [ ]:
#Testing - same algorithm as previous cell except it's for the test set
fq = open('test.txt', 'r')
test = fq.readlines()
fq.close()

probssub = []


count = 0
pairsindex = 0

stragglers = []

#print(uniwordprobs)

for t in test:
    if count == 800:
        count = 0
        if pairsindex < 24:
            pairsindex += 1
    sentence = re.compile('[\w{}À-ÿ|]+').findall(t)
    decide = "{" + pairs[pairsindex][0] + "|" + pairs[pairsindex][1] + "}"
    for s in sentence:
        if s == decide:
            prevword, nextword = None, None
            index = sentence.index(decide)
            if index > 0:
                prevword = sentence[index - 1]
            if index < (len(sentence) - 1):
                nextword = sentence[index + 1]
            alpha = 0
            beta = 0
            
            afront = 0.0
            aback = 0.0
            bfront = 0.0
            bback = 0.0

            afront = lamb * probs.get((prevword, pairs[pairsindex][0]), 0.0) \
            + lamb2 * uniwordprobs.get(pairs[pairsindex][0], 0.0)
                
            aback = lamb3 * probs.get((pairs[pairsindex][0], nextword), 0.0) \
            + lamb4 * alluniprobs.get(nextword, 0.0)

            bfront = lamb * probs.get((prevword, pairs[pairsindex][1]), 0.0) \
            + lamb2 * uniwordprobs.get(pairs[pairsindex][1], 0.0)
                
            bback = lamb3 * probs.get((pairs[pairsindex][1], nextword), 0.0) \
            + lamb4 * alluniprobs.get(nextword, 0.0)

            alpha =  afront * aback
            beta =  bfront * bback
            if alpha != 0 and beta != 0:
                probssub.append(alpha/(alpha + beta))
            else:
                uniprob = uniwordprobs.get(pairs[pairsindex][0], 0.0)
                probssub.append(uniprob)
                
                
            break
    count += 1
    
pairsindex = 0
       
print("Testing complete")

In [ ]:
#Writing guesses to csv file for submission
df = pandas.read_csv('sampleSubmission.csv')

count = 0
for index, row in df.iterrows():
    df.at[index, 'Expected'] = probssub[count]
    count += 1
        
df.to_csv('Submission.csv', index = False)

print("Writing complete")